In [1]:
""" Merge raw data into master table. 
-------------------------------------------------------------------------------

Author: Rutger Hofste
Date: 20181204
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""

TESTING = 0
SCRIPT_NAME = 'Y2018M12D04_RH_Master_Merge_Rawdata_GPD_V01'
OUTPUT_VERSION = 2

BQ_PROJECT_ID = "aqueduct30"
BQ_OUTPUT_DATASET_NAME = "aqueduct30v01"
BQ_OUTPUT_TABLE_NAME = "{}_v{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION).lower()


BQ_IN = {}


BQ_IN["Master"] = "y2018m12d06_rh_master_shape_v01_v01"


# Additional tables required

# area per smallest subunit
BQ_IN["area"] = "y2018m12d07_rh_process_area_bq_v01_v01"

# GADM countries 
BQ_IN["GADM36L01"] = "y2018m11d12_rh_gadm36_level1_rds_to_bq_v01_v01"


# FAO basins



# Area per aq30_id

# Physical Risk Quantity | QAN   -------------

# Baseline Water Stress | BWS
BQ_IN["BWS"] = "y2018m07d30_rh_gcs_to_bq_v01_v06"

# Baseline Water Depletion | BWD
BQ_IN["BWD"] = "y2018m07d30_rh_gcs_to_bq_v01_v06"

# Interannual Variability | IAV
BQ_IN["IAV"] = "y2018m07d31_rh_inter_av_cat_label_v01_v02"

# Seasonal Variability | SEV
BQ_IN["SEV"] = "y2018m08d02_rh_intra_annual_variability_cat_label_v01_v02"

# Riverine Flood Risk | RFR
BQ_IN["RFR"] = "y2018m12d04_rh_rfr_cfr_bq_v01_v01"

# Coastal Flood Risk | CFR
BQ_IN["CFR"] = "y2018m12d04_rh_rfr_cfr_bq_v01_v01"

# Drought Risk | DRR
BQ_IN["DRR"] = "y2018m09d28_rh_dr_cat_label_v01_v02"

# Groundwater Table Decline | GTD
BQ_IN["GTD"] = "y2018m09d03_rh_gws_cat_label_v01_v01"


# Physical Risk Quality | QAL -----------------

# Untreated Collected Wastewater | UCW
BQ_IN["UCW"] = "y2018m12d04_rh_ucw_bq_v01_v01"

# Coastal Eutrophication Potential | CEP
BQ_IN["CEP"] = "y2018m11d22_rh_icep_hybas6_cat_label_bq_v01_v03"

# Regulatory and Reputational Risk | RRR --------------

# Unimproved/no drinking water | UDW
BQ_IN["UDW"] = "y2018m12d05_rh_udw_bq_v01_v01"

# Unimproved/no sanitation | USA
BQ_IN["USA"] = "y2018m12d05_rh_usa_bq_v01_v01"

# RepRisk Index | RRI
BQ_IN["RRI"] = "y2018m12d05_rh_rri_bq_v01_v01"


ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION) 
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)



print("\nBQ_OUTPUT_DATASET_NAME: ", BQ_OUTPUT_DATASET_NAME,
      "\nBQ_OUTPUT_TABLE_NAME: ", BQ_OUTPUT_TABLE_NAME,
      "\ns3_output_path: ", s3_output_path,
      "\nec2_output_path:" , ec2_output_path)


BQ_OUTPUT_DATASET_NAME:  aqueduct30v01 
BQ_OUTPUT_TABLE_NAME:  y2018m12d04_rh_master_merge_rawdata_gpd_v01_v02 
s3_output_path:  s3://wri-projects/Aqueduct30/processData/Y2018M12D04_RH_Master_Merge_Rawdata_GPD_V01/output_V02/ 
ec2_output_path: /volumes/data/Y2018M12D04_RH_Master_Merge_Rawdata_GPD_V01/output_V02


In [2]:
BQ_IN

{'BWD': 'y2018m07d30_rh_gcs_to_bq_v01_v06',
 'BWS': 'y2018m07d30_rh_gcs_to_bq_v01_v06',
 'CEP': 'y2018m11d22_rh_icep_hybas6_cat_label_bq_v01_v03',
 'CFR': 'y2018m12d04_rh_rfr_cfr_bq_v01_v01',
 'DRR': 'y2018m09d28_rh_dr_cat_label_v01_v02',
 'GADM36L01': 'y2018m11d12_rh_gadm36_level1_rds_to_bq_v01_v01',
 'GTD': 'y2018m09d03_rh_gws_cat_label_v01_v01',
 'IAV': 'y2018m07d31_rh_inter_av_cat_label_v01_v02',
 'Master': 'y2018m12d06_rh_master_shape_v01_v01',
 'RFR': 'y2018m12d04_rh_rfr_cfr_bq_v01_v01',
 'RRI': 'y2018m12d05_rh_rri_bq_v01_v01',
 'SEV': 'y2018m08d02_rh_intra_annual_variability_cat_label_v01_v02',
 'UCW': 'y2018m12d04_rh_ucw_bq_v01_v01',
 'UDW': 'y2018m12d05_rh_udw_bq_v01_v01',
 'USA': 'y2018m12d05_rh_usa_bq_v01_v01',
 'area': 'y2018m12d07_rh_process_area_bq_v01_v01'}

In [3]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M12D11 UTC 19:53


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [4]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

In [5]:
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/.google.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "aqueduct30"
client = bigquery.Client(project=BQ_PROJECT_ID)

pd.set_option('display.max_columns', 500)

In [6]:
# Master Table

In [7]:
sql_master = """
SELECT
  aq30_id,
  string_id,
  pfaf_id,
  gid_1,
  aqid
FROM
  `{}.{}.{}`
ORDER BY
  aq30_id
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["Master"])

In [8]:
df_master = pd.read_gbq(query=sql_master,dialect="standard")

In [9]:
df_master.head()

,aq30_id,string_id,pfaf_id,gid_1,aqid
0,0,111011-EGY.11_1-3365,111011,EGY.11_1,3365
1,1,111011-EGY.15_1-3365,111011,EGY.15_1,3365
2,2,111011-EGY.15_1-None,111011,EGY.15_1,-9999
3,3,111011-None-3365,111011,-9999,3365
4,4,111011-None-None,111011,-9999,-9999


In [10]:
sql_area = """
SELECT
  string_id,
  area_km2
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["area"])

In [11]:
df_area = pd.read_gbq(query=sql_area,dialect="standard")

In [12]:
df_area.head()

,string_id,area_km2
0,111011-EGY.11_1-3365,4.223754
1,111011-EGY.15_1-3365,1846.012343
2,111011-EGY.15_1-None,30.526067
3,111011-None-3365,0.742712
4,111011-None-None,13.430995


In [13]:
df_area.shape

(68511, 2)

In [14]:
# GADM Cross Table

In [15]:
sql_gadm36l01 = """
SELECT
  gid_1,
  name_1,
  gid_0,
  name_0,
  varname_1,
  nl_name_1,
  type_1,
  engtype_1,
  cc_1,
  hasc_1
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["GADM36L01"])

In [16]:
df_gadm36l01 = pd.read_gbq(query=sql_gadm36l01,dialect="standard")

In [17]:
df_gadm36l01.head()

,gid_1,name_1,gid_0,name_0,varname_1,nl_name_1,type_1,engtype_1,cc_1,hasc_1
0,XAD.1_1,Akrotiri,XAD,Akrotiri and Dhekelia,Western Sovereign Base Area,None,Sovereign Base Area,Sovereign Base Area,None,None
1,XKO.1_1,Đakovica,XKO,Kosovo,None,None,Okrug,District,None,None
2,XKO.7_1,Uroševac,XKO,Kosovo,None,None,Okrug,District,None,None
3,XKO.4_1,Pećki,XKO,Kosovo,Peć,Пећки,Okrug,District,None,RS.PE
4,WSM.8_1,Satupa'itea,WSM,Samoa,Satupaitea,None,Itumalo,District,None,WS.SA


In [18]:
# BWS annual

In [19]:
sql_bws_annual = """
SELECT
  pfafid_30spfaf06,
  temporal_resolution,
  delta_id,
  year,
  waterstress_raw_dimensionless_coalesced,
  waterstress_score_dimensionless_coalesced,
  waterstress_category_dimensionless_coalesced,
  waterstress_label_dimensionless_coalesced
FROM
  `{}.{}.{}`
WHERE
  temporal_resolution = 'year'
  AND year = 2014
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["BWS"])

In [20]:
df_bws_annual = pd.read_gbq(query=sql_bws_annual,dialect="standard")

In [21]:
df_bws_annual = df_bws_annual.rename(columns={"pfafid_30spfaf06":"pfaf_id",
                                              "waterstress_raw_dimensionless_coalesced":"bws_raw",
                                              "waterstress_score_dimensionless_coalesced":"bws_score",
                                              "waterstress_category_dimensionless_coalesced":"bws_cat",
                                              "waterstress_label_dimensionless_coalesced":"bws_label"})

In [22]:
df_bws_annual.head()

,pfaf_id,temporal_resolution,delta_id,year,bws_raw,bws_score,bws_cat,bws_label
0,822478,year,-1,2014,0.055715,0.156149,0.0,Low
1,822295,year,-1,2014,0.072984,0.545647,0.0,Low
2,822340,year,-1,2014,0.088637,0.825980,0.0,Low
3,822320,year,-1,2014,0.091209,0.867252,0.0,Low
4,181207,year,-1,2014,0.062617,0.324630,0.0,Low


In [23]:
# Setting arid and low water use score to 5
df_bws_annual.bws_score.loc[df_bws_annual.bws_score == -1] = 5

/opt/anaconda3/envs/python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [24]:
# BWD annual

In [25]:
sql_bwd_annual = """
SELECT
  pfafid_30spfaf06,
  temporal_resolution,
  delta_id,
  year,
  waterdepletion_raw_dimensionless_coalesced,
  waterdepletion_score_dimensionless_coalesced,
  waterdepletion_category_dimensionless_coalesced,
  waterdepletion_label_dimensionless_coalesced
FROM
  `{}.{}.{}`
WHERE
  temporal_resolution = 'year'
  AND year = 2014
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["BWD"])

In [26]:
df_bwd_annual = pd.read_gbq(query=sql_bwd_annual,dialect="standard")

In [27]:
df_bwd_annual = df_bwd_annual.rename(columns={"pfafid_30spfaf06":"pfaf_id",
                                              "waterdepletion_raw_dimensionless_coalesced":"bwd_raw",
                                              "waterdepletion_score_dimensionless_coalesced":"bwd_score",
                                              "waterdepletion_category_dimensionless_coalesced":"bwd_cat",
                                              "waterdepletion_label_dimensionless_coalesced":"bwd_label"})

In [28]:
df_bwd_annual.drop(columns=["temporal_resolution",
                            "year",
                            "delta_id"],
                   inplace=True)

In [29]:
# Setting arid and low water use score to 5
df_bwd_annual.bwd_score.loc[df_bwd_annual.bwd_score == -1] = 5

/opt/anaconda3/envs/python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [30]:
# IAV 

In [31]:
sql_iav_annual = """
SELECT
  pfafid_30spfaf06,
  temporal_resolution,
  year,
  delta_id,
  iav_riverdischarge_m_coalesced,
  iav_riverdischarge_score_coalesced,
  iav_riverdischarge_category_coalesced,
  iav_riverdischarge_label_coalesced
FROM
  `{}.{}.{}`
WHERE
  temporal_resolution = 'year'
  AND year = 2014
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["IAV"])

In [32]:
df_iav_annual = pd.read_gbq(query=sql_iav_annual,dialect="standard")

In [33]:
df_iav_annual = df_iav_annual.rename(columns={"pfafid_30spfaf06":"pfaf_id",
                                              "iav_riverdischarge_m_coalesced":"iav_raw",
                                              "iav_riverdischarge_score_coalesced":"iav_score",
                                              "iav_riverdischarge_category_coalesced":"iav_cat",
                                              "iav_riverdischarge_label_coalesced":"iav_label"})

In [34]:
df_iav_annual.drop(columns=["temporal_resolution",
                            "year",
                            "delta_id"],
                   inplace=True)

In [35]:
# SEV

In [36]:
sql_sev = """
SELECT
  pfafid_30spfaf06,
  sv_riverdischarge_m_coalesced,
  sv_riverdischarge_score_coalesced,
  sv_riverdischarge_category_coalesced,
  sv_label_dimensionless_coalesced
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["SEV"])

In [37]:
df_sev = pd.read_gbq(query=sql_sev,dialect="standard")

In [38]:
df_sev = df_sev.rename(columns={"pfafid_30spfaf06":"pfaf_id",
                                "sv_riverdischarge_m_coalesced":"sev_raw",
                                "sv_riverdischarge_score_coalesced":"sev_score",
                                "sv_riverdischarge_category_coalesced":"sev_cat",
                                "sv_label_dimensionless_coalesced":"sev_label"})

In [39]:
# RFR

In [40]:
sql_rfr = """
SELECT
  pfaf_id,
  rfr_raw,
  rfr_score,
  rfr_cat,
  rfr_label
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["RFR"])

In [41]:
df_rfr = pd.read_gbq(query=sql_rfr,dialect="standard")

In [42]:
# CFR

In [43]:
sql_cfr = """
SELECT
  pfaf_id,
  cfr_raw,
  cfr_score,
  cfr_cat,
  cfr_label
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["CFR"])

In [44]:
df_cfr = pd.read_gbq(query=sql_cfr,dialect="standard")

In [45]:
# DRR

In [46]:
sql_drr = """
SELECT
  PFAF_ID,
  droughtrisk_score AS droughtrisk_raw,
  droughtrisk_score,
  droughtrisk_cat,
  droughtrisk_label
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["DRR"])

In [47]:
df_drr = pd.read_gbq(query=sql_drr,dialect="standard")

In [48]:
df_drr = df_drr.rename(columns={"PFAF_ID":"pfaf_id",
                                "droughtrisk_raw":"drr_raw",
                                "droughtrisk_score":"drr_score",
                                "droughtrisk_cat":"drr_cat",
                                "droughtrisk_label":"drr_label"})

In [49]:
# GTD

In [50]:
sql_gtd = """
SELECT
  AqID_spatial_unit,
  groundwatertabledecliningtrend_cmperyear,
  groundwatertabledecliningtrend_score,
  groundwatertabledecliningtrend_cat,
  groundwatertabledecliningtrend_label
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["GTD"])

In [51]:
df_gtd = pd.read_gbq(query=sql_gtd,dialect="standard")

In [52]:
df_gtd = df_gtd.rename(columns={"AqID_spatial_unit":"aqid",
                                "groundwatertabledecliningtrend_cmperyear":"gtd_raw",
                                "groundwatertabledecliningtrend_score":"gtd_score",
                                "groundwatertabledecliningtrend_cat":"gtd_cat",
                                "groundwatertabledecliningtrend_label":"gtd_label"})

In [53]:
# UCW

In [54]:
sql_ucw = """
SELECT
  adm0_a3,
  ucw_raw,
  ucw_score,
  ucw_cat,
  ucw_label
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["UCW"])

In [55]:
df_ucw = pd.read_gbq(query=sql_ucw,dialect="standard")

In [56]:
# CEP

In [57]:
sql_cep = """
SELECT
  pfaf_id,
  cep_raw,
  cep_score,
  cep_cat,
  cep_label
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["CEP"])

In [58]:
df_cep = pd.read_gbq(query=sql_cep,dialect="standard")

In [59]:
# UDW

In [60]:
sql_udw = """
SELECT
  pfaf_id,
  udw_raw,
  udw_score,
  udw_cat,
  udw_label
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["UDW"])

In [61]:
df_udw = pd.read_gbq(query=sql_udw,dialect="standard")

In [62]:
# USA

In [63]:
sql_usa = """
SELECT
  pfaf_id,
  usa_raw,
  usa_score,
  usa_cat,
  usa_label
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["USA"])

In [64]:
df_usa = pd.read_gbq(query=sql_usa,dialect="standard")

In [65]:
# RRI

In [66]:
sql_rri = """
SELECT
  adm0_a3,
  rri_raw,
  rri_score,
  rri_cat,
  rri_label
FROM
  `{}.{}.{}`
""".format(BQ_PROJECT_ID,BQ_OUTPUT_DATASET_NAME,BQ_IN["RRI"])

In [67]:
df_rri = pd.read_gbq(query=sql_rri,dialect="standard")

# Joining  Tables

In [68]:
df_master.head()

,aq30_id,string_id,pfaf_id,gid_1,aqid
0,0,111011-EGY.11_1-3365,111011,EGY.11_1,3365
1,1,111011-EGY.15_1-3365,111011,EGY.15_1,3365
2,2,111011-EGY.15_1-None,111011,EGY.15_1,-9999
3,3,111011-None-3365,111011,-9999,3365
4,4,111011-None-None,111011,-9999,-9999


In [69]:
df_master.shape

(68511, 5)

In [70]:
# Merge Area

In [71]:
df_merged = pd.merge(left=df_master,
                     right=df_area,
                     how="left",
                     left_on="string_id",
                     right_on="string_id",
                     validate="one_to_one")

In [72]:
# Merge GADM

In [73]:
df_gadm36l01.head()

,gid_1,name_1,gid_0,name_0,varname_1,nl_name_1,type_1,engtype_1,cc_1,hasc_1
0,XAD.1_1,Akrotiri,XAD,Akrotiri and Dhekelia,Western Sovereign Base Area,None,Sovereign Base Area,Sovereign Base Area,None,None
1,XKO.1_1,Đakovica,XKO,Kosovo,None,None,Okrug,District,None,None
2,XKO.7_1,Uroševac,XKO,Kosovo,None,None,Okrug,District,None,None
3,XKO.4_1,Pećki,XKO,Kosovo,Peć,Пећки,Okrug,District,None,RS.PE
4,WSM.8_1,Satupa'itea,WSM,Samoa,Satupaitea,None,Itumalo,District,None,WS.SA


In [74]:
df_merged = pd.merge(left=df_merged,
                     right=df_gadm36l01,
                     how="left",
                     left_on="gid_1",
                     right_on="gid_1",
                     validate="many_to_one")

In [75]:
# Merge BWS

In [76]:
df_merged = pd.merge(left=df_merged,
                     right=df_bws_annual,
                     how="left",
                     left_on="pfaf_id",
                     right_on="pfaf_id",
                     validate="many_to_one")

In [77]:
# Merge BWD

In [78]:
df_merged = pd.merge(left=df_merged,
                     right=df_bwd_annual,
                     how="left",
                     left_on="pfaf_id",
                     right_on="pfaf_id",
                     validate="many_to_one")

In [79]:
# Merge IAV

In [80]:
df_merged = pd.merge(left=df_merged,
                     right=df_iav_annual,
                     how="left",
                     left_on="pfaf_id",
                     right_on="pfaf_id",
                     validate="many_to_one")

In [81]:
# Merge SEV

In [82]:
df_merged = pd.merge(left=df_merged,
                     right=df_sev,
                     how="left",
                     left_on="pfaf_id",
                     right_on="pfaf_id",
                     validate="many_to_one")

In [83]:
# Merge RFR

In [84]:
df_merged = pd.merge(left=df_merged,
                     right=df_rfr,
                     how="left",
                     left_on="pfaf_id",
                     right_on="pfaf_id",
                     validate="many_to_one")

In [85]:
# Merge CFR

In [86]:
df_merged = pd.merge(left=df_merged,
                     right=df_cfr,
                     how="left",
                     left_on="pfaf_id",
                     right_on="pfaf_id",
                     validate="many_to_one")

In [87]:
# Merge DRR

In [88]:
df_merged = pd.merge(left=df_merged,
                     right=df_drr,
                     how="left",
                     left_on="pfaf_id",
                     right_on="pfaf_id",
                     validate="many_to_one")

In [89]:
# Merge GTD

In [90]:
df_merged = pd.merge(left=df_merged,
                     right=df_gtd,
                     how="left",
                     left_on="aqid",
                     right_on="aqid",
                     validate="many_to_one")

In [91]:
# UCW

In [92]:
df_merged = pd.merge(left=df_merged,
                     right=df_ucw,
                     how="left",
                     left_on="gid_0",
                     right_on="adm0_a3",
                     validate="many_to_one")

In [93]:
df_merged.drop(columns=["adm0_a3"],
               inplace=True)

In [94]:
df_merged.head()

,aq30_id,string_id,pfaf_id,gid_1,aqid,area_km2,name_1,gid_0,name_0,varname_1,nl_name_1,type_1,engtype_1,cc_1,hasc_1,temporal_resolution,delta_id,year,bws_raw,bws_score,bws_cat,bws_label,bwd_raw,bwd_score,bwd_cat,bwd_label,iav_raw,iav_score,iav_cat,iav_label,sev_raw,sev_score,sev_cat,sev_label,rfr_raw,rfr_score,rfr_cat,rfr_label,cfr_raw,cfr_score,cfr_cat,cfr_label,drr_raw,drr_score,drr_cat,drr_label,gtd_raw,gtd_score,gtd_cat,gtd_label,ucw_raw,ucw_score,ucw_cat,ucw_label
0,0,111011-EGY.11_1-3365,111011,EGY.11_1,3365,4.223754,Al Qahirah,EGY,Egypt,Cairo|El Cairo|El Qahira|Le Caire,None,Muhafazah,Governorate,None,EG.QH,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",-9999.0,-9999.0,-9999.0,NoData,0.843958,-9998.0,-9998.0,Insignificant Trend,0.6139,2.046333,2.0,Medium to high (0.6 to 90%)
1,1,111011-EGY.15_1-3365,111011,EGY.15_1,3365,1846.012343,As Suways,EGY,Egypt,El Suweiz|Es Suweis|Suez,محافظة السويس,Muhafazah,Governorate,None,EG.SW,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",-9999.0,-9999.0,-9999.0,NoData,0.843958,-9998.0,-9998.0,Insignificant Trend,0.6139,2.046333,2.0,Medium to high (0.6 to 90%)
2,2,111011-EGY.15_1-None,111011,EGY.15_1,-9999,30.526067,As Suways,EGY,Egypt,El Suweiz|Es Suweis|Suez,محافظة السويس,Muhafazah,Governorate,None,EG.SW,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",-9999.0,-9999.0,-9999.0,NoData,NaN,NaN,NaN,NaN,0.6139,2.046333,2.0,Medium to high (0.6 to 90%)
3,3,111011-None-3365,111011,-9999,3365,0.742712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",-9999.0,-9999.0,-9999.0,NoData,0.843958,-9998.0,-9998.0,Insignificant Trend,NaN,NaN,NaN,NaN
4,4,111011-None-None,111011,-9999,-9999,13.430995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",-9999.0,-9999.0,-9999.0,NoData,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
# CEP

In [96]:
df_merged = pd.merge(left=df_merged,
                     right=df_cep,
                     how="left",
                     left_on="pfaf_id",
                     right_on="pfaf_id",
                     validate="many_to_one")

In [97]:
# UDW

In [98]:
df_merged = pd.merge(left=df_merged,
                     right=df_udw,
                     how="left",
                     left_on="pfaf_id",
                     right_on="pfaf_id",
                     validate="many_to_one")

In [99]:
# USA

In [100]:
df_merged = pd.merge(left=df_merged,
                     right=df_usa,
                     how="left",
                     left_on="pfaf_id",
                     right_on="pfaf_id",
                     validate="many_to_one")

In [101]:
df_merged.head()

,aq30_id,string_id,pfaf_id,gid_1,aqid,area_km2,name_1,gid_0,name_0,varname_1,nl_name_1,type_1,engtype_1,cc_1,hasc_1,temporal_resolution,delta_id,year,bws_raw,bws_score,bws_cat,bws_label,bwd_raw,bwd_score,bwd_cat,bwd_label,iav_raw,iav_score,iav_cat,iav_label,sev_raw,sev_score,sev_cat,sev_label,rfr_raw,rfr_score,rfr_cat,rfr_label,cfr_raw,cfr_score,cfr_cat,cfr_label,drr_raw,drr_score,drr_cat,drr_label,gtd_raw,gtd_score,gtd_cat,gtd_label,ucw_raw,ucw_score,ucw_cat,ucw_label,cep_raw,cep_score,cep_cat,cep_label,udw_raw,udw_score,udw_cat,udw_label,usa_raw,usa_score,usa_cat,usa_label
0,0,111011-EGY.11_1-3365,111011,EGY.11_1,3365,4.223754,Al Qahirah,EGY,Egypt,Cairo|El Cairo|El Qahira|Le Caire,None,Muhafazah,Governorate,None,EG.QH,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",-9999.0,-9999.0,-9999.0,NoData,0.843958,-9998.0,-9998.0,Insignificant Trend,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%)
1,1,111011-EGY.15_1-3365,111011,EGY.15_1,3365,1846.012343,As Suways,EGY,Egypt,El Suweiz|Es Suweis|Suez,محافظة السويس,Muhafazah,Governorate,None,EG.SW,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",-9999.0,-9999.0,-9999.0,NoData,0.843958,-9998.0,-9998.0,Insignificant Trend,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%)
2,2,111011-EGY.15_1-None,111011,EGY.15_1,-9999,30.526067,As Suways,EGY,Egypt,El Suweiz|Es Suweis|Suez,محافظة السويس,Muhafazah,Governorate,None,EG.SW,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",-9999.0,-9999.0,-9999.0,NoData,NaN,NaN,NaN,NaN,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%)
3,3,111011-None-3365,111011,-9999,3365,0.742712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",-9999.0,-9999.0,-9999.0,NoData,0.843958,-9998.0,-9998.0,Insignificant Trend,NaN,NaN,NaN,NaN,0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%)
4,4,111011-None-None,111011,-9999,-9999,13.430995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",-9999.0,-9999.0,-9999.0,NoData,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%)


In [102]:
# RRI

In [103]:
df_merged = pd.merge(left=df_merged,
                     right=df_rri,
                     how="left",
                     left_on="gid_0",
                     right_on="adm0_a3",
                     validate="many_to_one")

In [104]:
df_merged.drop(columns=["adm0_a3"],
               inplace=True)

In [105]:
# The dataframe has None's and NoData Values. 
# Creating two versions. One with None's and one with NoData values. 

In [106]:
df_merged_nones = df_merged.replace(to_replace=[-9999,-9998,-9999.0,-9998.0,"-9999","NoData"],value= np.nan)

In [107]:
df_merged_nones.head()

,aq30_id,string_id,pfaf_id,gid_1,aqid,area_km2,name_1,gid_0,name_0,varname_1,nl_name_1,type_1,engtype_1,cc_1,hasc_1,temporal_resolution,delta_id,year,bws_raw,bws_score,bws_cat,bws_label,bwd_raw,bwd_score,bwd_cat,bwd_label,iav_raw,iav_score,iav_cat,iav_label,sev_raw,sev_score,sev_cat,sev_label,rfr_raw,rfr_score,rfr_cat,rfr_label,cfr_raw,cfr_score,cfr_cat,cfr_label,drr_raw,drr_score,drr_cat,drr_label,gtd_raw,gtd_score,gtd_cat,gtd_label,ucw_raw,ucw_score,ucw_cat,ucw_label,cep_raw,cep_score,cep_cat,cep_label,udw_raw,udw_score,udw_cat,udw_label,usa_raw,usa_score,usa_cat,usa_label,rri_raw,rri_score,rri_cat,rri_label
0,0,111011-EGY.11_1-3365,111011.0,EGY.11_1,3365.0,4.223754,Al Qahirah,EGY,Egypt,Cairo|El Cairo|El Qahira|Le Caire,None,Muhafazah,Governorate,None,EG.QH,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,NaN,0.843958,NaN,NaN,Insignificant Trend,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),58.0,2.8,2.0,Medium to high (50 to 60%)
1,1,111011-EGY.15_1-3365,111011.0,EGY.15_1,3365.0,1846.012343,As Suways,EGY,Egypt,El Suweiz|Es Suweis|Suez,محافظة السويس,Muhafazah,Governorate,None,EG.SW,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,NaN,0.843958,NaN,NaN,Insignificant Trend,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),58.0,2.8,2.0,Medium to high (50 to 60%)
2,2,111011-EGY.15_1-None,111011.0,EGY.15_1,NaN,30.526067,As Suways,EGY,Egypt,El Suweiz|Es Suweis|Suez,محافظة السويس,Muhafazah,Governorate,None,EG.SW,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6139,2.046333,2.0,Medium to high (0.6 to 90%),0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),58.0,2.8,2.0,Medium to high (50 to 60%)
3,3,111011-None-3365,111011.0,NaN,3365.0,0.742712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,NaN,0.843958,NaN,NaN,Insignificant Trend,NaN,NaN,NaN,NaN,0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),NaN,NaN,NaN,NaN
4,4,111011-None-None,111011.0,NaN,NaN,13.430995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,year,-1.0,2014.0,2.318042,5.0,4.0,Extremely High,0.987061,4.948243,4.0,Extremely High,1.035414,4.141657,4.0,Extremely High,0.962396,2.887187,2.0,Medium - High,0.074383,4.180674,4.0,Extremely High (more than 1 in 100),0.0,0.0,0.0,"Low (0 to 9 in 1,000,000)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.25,1.0,Low to medium (-5 to 0),0.011222,0.0,0.0,Low (< 2.5%),0.025333,1.019067,1.0,Low to medium (2.5 to 5%),NaN,NaN,NaN,NaN


In [108]:
destination_path_s3 = "{}/{}.pkl".format(ec2_output_path,SCRIPT_NAME)

In [109]:
df_merged_nones.to_pickle(destination_path_s3)

In [110]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive

upload: ../../../../data/Y2018M12D04_RH_Master_Merge_Rawdata_GPD_V01/output_V02/Y2018M12D04_RH_Master_Merge_Rawdata_GPD_V01.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M12D04_RH_Master_Merge_Rawdata_GPD_V01/output_V02/Y2018M12D04_RH_Master_Merge_Rawdata_GPD_V01.pkl


In [111]:
destination_table = "{}.{}".format(BQ_OUTPUT_DATASET_NAME,BQ_OUTPUT_TABLE_NAME)

In [112]:
destination_table

'aqueduct30v01.y2018m12d04_rh_master_merge_rawdata_gpd_v01_v02'

In [113]:
df_merged_nones.to_gbq(destination_table=destination_table,
                       project_id=BQ_PROJECT_ID,
                       chunksize=10000,
                       if_exists="replace")

7it [01:00,  8.71s/it]


In [114]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:02:23.445872


Previous runs:   
0:02:28.099761
